In [2]:
import os
import openai
import dotenv
from llama_index.core import SimpleDirectoryReader


dotenv.load_dotenv()

openai.api_key = os.getenv("OPENAI_API_KEY")

documents = SimpleDirectoryReader(
    input_files=["./eBook-How-to-Build-a-Career-in-AI.pdf"]
).load_data()

documents[0]

Document(id_='b25a6156-9fd1-4439-ae82-ec812da1b63f', embedding=None, metadata={'page_label': '1', 'file_name': 'eBook-How-to-Build-a-Career-in-AI.pdf', 'file_path': 'eBook-How-to-Build-a-Career-in-AI.pdf', 'file_type': 'application/pdf', 'file_size': 3717673, 'creation_date': '2025-12-02', 'last_modified_date': '2025-12-02'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text_resource=MediaResource(embeddings=None, data=None, text='PAGE 1\nFounder, DeepLearning.AI\nCollected Insights\nfrom Andrew Ng\nHow to \nBuild\nYour\nCareer\nin AI\nA Simple Guide\n', path=None, url=None, mimetype=None), image_resource=None, audio_resource=None, video_resource=None, text_template='{metadata_str}\n\n{content}')

In [3]:
from llama_index.core import Document, VectorStoreIndex
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings
from llama_index.llms.openai import OpenAI

Settings.llm = OpenAI(model="gpt-4o-mini", temperature=0.1)

Settings.embed_model = HuggingFaceEmbedding(
    model_name="BAAI/bge-small-en-v1.5"
)

document = Document(text="\n\n".join([doc.text for doc in documents]))
index = VectorStoreIndex.from_documents([document], llm=Settings.llm, embed_model=Settings.embed_model)

query_engine = index.as_query_engine()
response = query_engine.query("What is the main idea of the book?")
print(response)



The main idea of the book is to provide a comprehensive guide for individuals looking to build a successful career in artificial intelligence (AI). It emphasizes the importance of learning foundational technical skills, working on meaningful projects to enhance those skills, and effectively navigating the job search process. The book also discusses the evolving significance of coding, particularly in AI, and offers strategies for overcoming challenges such as imposter syndrome while building a portfolio that demonstrates skill progression.


##### Evaluation setup using TruLens

In [4]:
eval_questions = []
with open("./eval_questions.txt", "r") as file:
    eval_questions = file.readlines()

eval_questions = [question.strip() for question in eval_questions]


In [5]:
new_q = "How much math should I know to build a career in AI?"
eval_questions.append(new_q)

In [6]:
from trulens_eval import Tru
from utils import get_prebuilt_trulens_recorder
import os

os.environ["TRULENS_OTEL_TRACING"] = "0"


tru = Tru()
tru.reset_database()

tru_recorder = get_prebuilt_trulens_recorder(query_engine, "Direct Query Evaluation")

with tru_recorder as recording:
    for question in eval_questions:
        response = query_engine.query(question)

records, feedbacks = tru.get_records_and_feedback(app_ids=[])
records.head()
tru.run_dashboard()

Updating app_name and app_version in apps table: 0it [00:00, ?it/s]
Updating app_id in records table: 0it [00:00, ?it/s]
Updating app_json in apps table: 0it [00:00, ?it/s]

instrumenting <class 'llama_index.embeddings.huggingface.base.HuggingFaceEmbedding'> for base <class 'llama_index.embeddings.huggingface.base.HuggingFaceEmbedding'>
instrumenting <class 'llama_index.embeddings.huggingface.base.HuggingFaceEmbedding'> for base <class 'llama_index.core.embeddings.multi_modal_base.MultiModalEmbedding'>
instrumenting <class 'llama_index.embeddings.huggingface.base.HuggingFaceEmbedding'> for base <class 'llama_index.core.base.embeddings.base.BaseEmbedding'>
instrumenting <class 'llama_index.embeddings.huggingface.base.HuggingFaceEmbedding'> for base <class 'llama_index.core.schema.TransformComponent'>
instrumenting <class 'llama_index.embeddings.huggingface.base.HuggingFaceEmbedding'> for base <class 'llama_index.core.schema.BaseComponent'>
instrumenting <class 'llama_index.embeddings.huggingface.base.HuggingFaceEmbedding'> for base <class 'pydantic.main.BaseModel'>
instrumenting <class 'llama_index.embeddings.huggingface.base.HuggingFaceEmbedding'> for base

instrumenting <class 'llama_index.core.base.llms.types.LLMMetadata'> for base <class 'llama_index.core.base.llms.types.LLMMetadata'>
instrumenting <class 'llama_index.core.base.llms.types.LLMMetadata'> for base <class 'pydantic.main.BaseModel'>
instrumenting <class 'llama_index.core.base.llms.types.LLMMetadata'> for base <class 'object'>
instrumenting <class 'tuple'> for base <class 'tuple'>
instrumenting <class 'tuple'> for base <class 'object'>
instrumenting <class 'llama_index.core.indices.prompt_helper.PromptHelper'> for base <class 'llama_index.core.indices.prompt_helper.PromptHelper'>
instrumenting <class 'llama_index.core.indices.prompt_helper.PromptHelper'> for base <class 'llama_index.core.schema.BaseComponent'>
instrumenting <class 'llama_index.core.indices.prompt_helper.PromptHelper'> for base <class 'pydantic.main.BaseModel'>
instrumenting <class 'llama_index.core.indices.prompt_helper.PromptHelper'> for base <class 'object'>
instrumenting <class 'llama_index.core.response_

ModuleNotFoundError: These package is required for evaluating feedback using OpenAI:

    trulens-providers-openai

You should be able to install it with pip:

    ```bash
    pip install "trulens-providers-openai>=1.0.0"
    ```